In [7]:
from pathlib import Path
from typing import Iterable, List, Tuple
import numpy as np
import h5py
import cv2
from PIL import Image
import datetime as dt

import glob

In [8]:
def get_coords(state, indices, m_points):
    i, j = indices[state['idx']]
    x_coords = [
        m_points[i, j]['x'],
        m_points[i, j+1]['x'],
        m_points[i+1, j+1]['x'],
        m_points[i+1, j]['x'],
        m_points[i, j]['x']  # close the loop
    ]
    y_coords = [
        m_points[i, j]['y'],
        m_points[i, j+1]['y'],
        m_points[i+1, j+1]['y'],
        m_points[i+1, j]['y'],
        m_points[i, j]['y']
    ]
    return x_coords, y_coords

def get_matrix_points(crossing_points_path='data/crossings_points.pkl'):
    with open(crossing_points_path, 'rb') as f:
        crossing_points = pickle.load(f)

    dt = np.dtype([('x', 'i4'), ('y', 'i4')])
    m_points = np.zeros((18, 18), dtype=dt)
    i = 0 
    j = 0
    for p in crossing_points:
        m_points[i, j] = p
        j += 1
        if j%18 == 0:
            j = 0
            i += 1
    return m_points

In [2]:
class WebcamH5Writer:
    def __init__(
        self,
        image_paths: Iterable[str | Path],
        m_points: np.ndarray,
        h5_path: str = "webcam_images_color.h5",
        rows: int = 18,
        cols: int = 18,
        compression: str = "gzip",
        compression_opts: int = 4
    ):
        self.image_paths = [Path(p) for p in image_paths]
        self.m_points = m_points
        self.h5_path = h5_path
        self.rows = rows
        self.cols = cols
        self.N_mirrors = (rows - 1) * (cols - 1)
        self.compression = compression
        self.compression_opts = compression_opts

    def _mirror_indices(self) -> List[Tuple[int, int]]:
        # Mirrors are the (rows-1) x (cols-1) cells in the grid
        return [(i, j) for i in range(self.rows - 1) for j in range(self.cols - 1)]

    def get_mirror_polygon(self, idx: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Return x and y vertex arrays for mirror 'idx' using your existing get_coords().
        """
        indices = self._mirror_indices()
        state = {'idx': idx}
        x, y = get_coords(state, indices, self.m_points)  # <- your function
        x = np.asarray(x, dtype=np.int32)
        y = np.asarray(y, dtype=np.int32)
        return x, y

    @staticmethod
    def extract_polygon_region_color(img_rgb: np.ndarray, x_coords: np.ndarray, y_coords: np.ndarray) -> np.ndarray:
        """
        Extract a polygonal region from an RGB image (H, W, 3) using a mask, then crop to bounding rect.
        """
        if img_rgb.ndim != 3 or img_rgb.shape[2] != 3:
            raise ValueError("img_rgb must be an RGB image with shape (H, W, 3).")
        if img_rgb.dtype != np.uint8:
            img_rgb = img_rgb.astype(np.uint8)

        pts = np.stack([x_coords, y_coords], axis=1).astype(np.int32)  # (K, 2)
        pts = pts.reshape((-1, 1, 2))  # (K, 1, 2) for cv2

        H, W, _ = img_rgb.shape
        mask = np.zeros((H, W), dtype=np.uint8)
        cv2.fillPoly(mask, [pts], color=255)

        # Single-channel mask works with 3-channel bitwise_and
        masked = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

        x, y, w, h = cv2.boundingRect(pts)
        cropped = masked[y:y + h, x:x + w]

        return cropped

    def get_mirror(self, idx: int, image_rgb: np.ndarray) -> np.ndarray:
        x, y = self.get_mirror_polygon(idx)
        mirror = self.extract_polygon_region_color(image_rgb, x, y)
        return mirror


In [4]:
! pwd

/media/pgliwny/ADATA HD3302/Computer_vision_for_MAGIC/prepare_images


In [5]:
WEBCAM_IMG_PATH = "/media/pgliwny/ADATA HD332/Computer_Vision_system/data/MAGIC/webcam/"
IMAGE_LIST = glob.glob(os.path.join(WEBCAM_IMG_PATH,"image_2024*1200*"))
print(IMAGE_LIST[:3])
print(len(IMAGE_LIST))

NameError: name 'glob' is not defined

In [3]:
writer = WebcamH5Writer()

TypeError: WebcamH5Writer.__init__() missing 2 required positional arguments: 'image_paths' and 'm_points'